## IMPORT BIBLIOTHÈQUE

In [207]:
# Importation des bibliothèques pour la manipulation et l'analyse des données.
import pandas as pd
import re
from dateutil import parser
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt

## LECTURE CSV

In [149]:
# Chargement du jeu de données depuis le fichier CSV. Paramètre low_memory=False utilisé pour éviter les erreurs de type
df = pd.read_csv("babyfoot_dataset.csv", low_memory=False) # Chargement du CSV qui est dans le même dossier que le notebook

# Un premier nettoyage est immédiatement appliquée pour supprimer les caractères (�) liés à d'éventuels problèmes d'encodage
df_clean = df.apply(lambda col: col.astype(str).str.replace('�', '', regex=False))


## NETTOYAGE DES DONNÉS

In [150]:
# Les colonnes jugées non pertinentes pour l'analyse sont supprimées:
# - attendance_count : Nombre de personnes présentes
# - music_playing : Musique de fond ou playlist
# - misc : Champ supplémentaire aléatoire
# - duplicate_flag : Indicateur pour les lignes en double suspectées
# - ping_ms : Ping/latence
# - recorded_by : Qui a enregistré le jeu (appareil/personne/bot)
# - mood : Humeur du joueur
df_clean = df_clean.drop(['attendance_count', 'music_playing', 'misc', 'duplicate_flag', 'ping_ms', 'recorded_by', 'mood'], axis=1)

In [151]:
# Suppression des lignes dupliquées
df_clean = df_clean.drop_duplicates()

#### Nettoyage des colonnes 'final_score_red' et 'final_score_blue'

In [152]:
# Transformation de toutes les valeurs de la colonne final_score_red et final_score_blue en string
df_clean['final_score_red'] = df_clean['final_score_red'].astype(str)
df_clean['final_score_blue'] = df_clean['final_score_blue'].astype(str)

# Remplacement de tout ce qui n’est pas un chiffre (0-9) ni un tiret (-) par '' grâce à une regex
df_clean['final_score_red'] = df_clean['final_score_red'].str.replace(r'[^0-9-]', '', regex=True)

# Remplacement de tout ce qui n’est pas un chiffre (0-9) par '' grâce à une regex
df_clean['final_score_blue'] = df_clean['final_score_blue'].str.replace(r'[^0-9]', '', regex=True)

In [153]:
# Remplacement des chaînes vides dans la colonne 'final_score_blue' par NA et convertir en Int64
df_clean['final_score_blue'] = df_clean['final_score_blue'].replace('', pd.NA).astype('Int64')

# Extraction du chiffre après '-' dans 'final_score_red' qui correspond au score de l'équipe bleu et convertir en Int64
df_clean['final_score_red_after_dash'] = df_clean['final_score_red'].str.extract(r'-(\d+)')[0].astype('Int64')

# Création de la nouvelle colonne pour le score de l'équipe bleu et on remplit les valeurs NA par les valeurs de final_score_red_after_dash
df_clean['blue_score_final'] = df_clean['final_score_blue'].fillna(df_clean['final_score_red_after_dash']).copy()

# Remplacement des valeurs dans la colonne 'final_score_red' en ne gardant que le chiffre avant le '-' grâce à une regex
df_clean['final_score_red'] = df_clean['final_score_red'].str.replace(r'\s*-\s*\d+', '', regex=True).astype('Int64')

In [154]:
# Suppression des anciennes colonne 'final_score_blue' et 'final_score_red_after_dash'
df_clean = df_clean.drop('final_score_red_after_dash', axis=1)
df_clean = df_clean.drop('final_score_blue', axis=1)

In [155]:
# Renommage de la nouvelle colonne 'blue_score_final' en 'final_score_blue'
df_clean.rename(columns={'blue_score_final': 'final_score_blue',}, inplace=True)

#### Normalisation de la colonne 'winner'

In [156]:
# Création d'une fonction pour assigner à qui donner la victoire en fonction des colonnes de score
def decide_winner(row):
    if row['final_score_red'] > row['final_score_blue']:
        return 'red'
    elif row['final_score_red'] < row['final_score_blue']:
        return 'blue'
    else:
        return 'tie'  # égalité

# Application de la fonction decide_winner
df_clean['winner'] = df_clean.apply(decide_winner, axis=1)

#### Normalisation de la colonne 'rating_raw'

In [157]:
# Mapping des nombres écrits en chiffres
words_to_numbers = {
    'four': 4,
    'five': 5,
}

# Mapping des emojis en chiffres
emoji_to_number = {
    '😡': 1,
    '👍': 2,
    '🙂': 4,
    '⭐⭐⭐': 3
}

# Remplacer les emojis par leur chiffre grâce à au mapping
df_clean['rating_raw'] = df_clean['rating_raw'].replace(emoji_to_number)

# Convertir les nombres écrits en minuscules puis remplacer par les chiffres grâce au mapping
df_clean['rating_raw'] = df_clean['rating_raw'].str.lower().replace(words_to_numbers)

# Convertir la colonne 'rating_raw' en Int64
df_clean['rating_raw'] = pd.to_numeric(df_clean['rating_raw'], errors='coerce').astype('Int64')

#### Normalisation de la colonne 'location'

In [158]:
# Normalisation du texte pour avoir le même texte pour une valeur identique
df_clean['location'] = df_clean['location'].replace('Ynov Tls', 'Ynov Toulouse')

#### Normalisation de la colonne 'player_age'

In [159]:
# Normalisation du texte pour avoir le même texte pour une valeur identique
df_clean['player_age'] = df_clean['player_age'].replace('twenty', '20')
df_clean['player_age'] = df_clean['player_age'].replace('21 yrs', '21')

#### Normalisation de la colonne 'player_role'

In [160]:
# Normalisation du texte pour avoir le même texte pour une valeur identique
df_clean['player_role'] = df_clean['player_role'].replace(['def', 'defense','defence'], 'Defense')
df_clean['player_role'] = df_clean['player_role'].replace(['attack', 'attck','ATTACK'], 'Attack')

#### Normalisation de la colonne 'player_goals'

In [161]:
# Normalisation du texte pour avoir le même texte pour une valeur identique
df_clean['player_goals'] = df_clean['player_goals'].replace('one', '1')
df_clean['player_goals'] = df_clean['player_goals'].replace('ten', '10')
df_clean['player_goals'] = df_clean['player_goals'].replace('two', '2')
df_clean['player_goals'] = df_clean['player_goals'].replace('three', '3')

#### Normalisation de la colonne 'team_color'

In [162]:
# Normalisation du texte pour avoir le même texte pour une valeur identique
df_clean['team_color'] = df_clean['team_color'].replace(['R', '🔴', 'red'], 'Red')
df_clean['team_color'] = df_clean['team_color'].replace(['B', '🔵', 'blue'], 'Blue')

In [163]:
# Séparation du nom et prénom dans 2 colonnes différente supprime les espaces autour
df_clean['name'] = df_clean['player_canonical_name'].str.strip().str.split().str[1]
df_clean['surname'] = df_clean['player_canonical_name'].str.strip().str.split().str[0]


#### Normalisation de la colonne 'game_date'


In [164]:
# Création d'une nouvelle colonne pour stocker les valeurs nettoyé
df_clean['game_date_cleaned'] = (
    df_clean['game_date']
    # 1. Assurer que la colonne est de type string pour les manipulations textuelles.
    .astype(str)
    # 2. Supprimer les espaces en début et fin de chaîne.
    .str.strip()
    # 3. Supprimer les indicateurs (ex: "1st", "2nd") pour ne garder que le chiffre.
    .str.replace(r'(\d{1,2})(st|nd|rd|th)', r'\1', regex=True)
    # 4. Uniformiser les séparateurs de date (virgules, points, slashs) en tirets.
    .str.replace(r'[,./]', '-', regex=True)
    # 5. Remplacer les espaces multiples par un seul espace pour un formatage propre.
    .str.replace(r'\s+', ' ', regex=True)
)

In [165]:
# Conversion de la colonne 'game_date_cleaned' nettoyées en format datetime.
df_clean['game_date_parsed'] = pd.to_datetime(df_clean['game_date_cleaned'], errors='coerce', dayfirst=True)

In [166]:
# Fonction de parsing de date pour gérer les formats variés.
def parse_flexible(date_str):
    try:
        return parser.parse(date_str, dayfirst=True, fuzzy=True)
    except Exception:
        return pd.NaT

# Application de la fonction de parsing à la colonne 'game_date_parsed'.
df_clean['game_date_parsed'] = df_clean['game_date_cleaned'].apply(parse_flexible)

In [167]:
# Suppression de l'ancienne colonne 'game_date' et de la colonne temporaire 'game_date_cleaned'
df_clean = df_clean.drop(['game_date', 'game_date_cleaned'], axis=1)

In [168]:
# Renommage de la nouvelle colonne 'game_date_parsed' en 'game_date'
df_clean.rename(columns={'game_date_parsed': 'game_date',}, inplace=True)

#### Normalisation de la colonne 'game_duration'


In [169]:
# Fonction de nettoyage de durée pour gérer les formats variés.
def nettoyer_game_duration(valeur):
    valeur = str(valeur).strip().lower()

    # hh:mm:ss → convertir en secondes
    if re.match(r'^\d{1,2}:\d{1,2}:\d{1,2}$', valeur):
        h, m, s = map(int, valeur.split(':'))
        total_seconds = h * 3600 + m * 60 + s

    # mm:ss → convertir en secondes
    elif re.match(r'^\d{1,2}:\d{1,2}$', valeur):
        m, s = map(int, valeur.split(':'))
        total_seconds = m * 60 + s

    # décimal → ex: 8.46 = 8 min 46 sec
    elif re.match(r'^\d+\.\d+$', valeur):
        m, s = valeur.split('.')
        total_seconds = int(m) * 60 + int(s)

    # texte → ex: "12min", "5m", "46s"
    elif 'min' in valeur or 'm' in valeur or 's' in valeur:
        match = re.findall(r'\d+', valeur)
        if len(match) == 2:
            m, s = map(int, match)
            total_seconds = m * 60 + s
        elif len(match) == 1:
            if 's' in valeur:
                total_seconds = int(match[0])
            else:
                total_seconds = int(match[0]) * 60
        else:
            return None

    # entier brut → ex: "94" secondes
    elif valeur.isdigit():
        total_seconds = int(valeur)

    else:
        return None

    # Format final en mm:ss
    minutes = total_seconds // 60
    secondes = total_seconds % 60
    return f"{minutes:02d}:{secondes:02d}"

In [170]:
# Application de la fonction de nettoyage de durée à la colonne 'game_duration'.
df_clean['game_duration'] = df_clean['game_duration'].apply(nettoyer_game_duration)

#### Normalisation de la colonne 'season'


In [171]:
# Normalisation du texte pour avoir le même texte pour une valeur identique
df_clean['season'] = df_clean['season'].replace('2025 Season', '2024/2025')
df_clean['season'] = df_clean['season'].replace('s24/25', '2024/2025')
df_clean['season'] = df_clean['season'].replace('Season 24-25', '2024/2025')

#### Normalisation de la colonne 'possession_time'

In [172]:
# Fonction de conversion de durée pour gérer les formats variés.
def convert_to_minutes(val):
    try:
        val = str(val).strip().lower()

        # Format "mm:ss" →
        if re.match(r'^\d{1,2}:\d{2}$', val):
            m, s = map(int, val.split(':'))
            return round(m + s / 60, 2)

        # Format "mm.ssmin" →
        if 'min' in val and '.' in val:
            val = val.replace('min', '')
            minutes = float(val)
            return round(minutes, 2)

        # Format "entier sans ,ou ." → secondes
        if val.isdigit():
            seconds = int(val)
            return round(seconds / 60, 2)

        # Format "6.88min" or "2.44min"
        if 'min' in val:
            val = val.replace('min', '')
            minutes = float(val)
            return round(minutes, 2)

        return None
    except:
        return None

In [173]:
# Application de la fonction de conversion à la colonne 'possession_time'.
df_clean['possession_time'] = df_clean['possession_time'].apply(convert_to_minutes)

In [174]:
# Remplacer les valeurs nulles par 0
df_clean['possession_time'] = df_clean['possession_time'].fillna(0)

#### Export du CSV


In [175]:
# Export du CSV nettoyé pour la création du rapport avec Power BI
df_clean.to_csv('game_dates_cleaned.csv', index=False)

## PRÉPARATION DES TABLES POUR INJECTER LES DONNÉES EN BDD


In [176]:
# Configuration pour se connecter à la BDD
POSTGRES_USER = "babyfoot"
POSTGRES_PASSWORD = "azerty"
POSTGRES_DB = "db1-hackathon"
POSTGRES_IP = "172.20.10.3"
POSTGRES_PORT = "5432"

# Chaîne de connexion
engine = create_engine(
    f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_IP}:{POSTGRES_PORT}/{POSTGRES_DB}"
)

#### Traitement de la table reservation

In [177]:
# Préparation de la table reservation
reservation = df_clean[['player_id', 'table_id']].copy()
reservation['start_time'] = None
reservation['end_time'] = None
reservation['status_reservation'] = None

In [ ]:
# Envoie du dataframe reservation vers la table de la BDD
reservation.to_sql(
    'reservation',
    con=engine,
    if_exists='append',
    index=False)

#### Traitement de la table game_player

In [ ]:
# Préparation de la table game_player
game_player = df_clean[['game_id', 'player_id', 'player_goals', 'player_own_goals', 'player_assists', 'player_saves', 'possession_time', 'player_role', 'player_comment', 'team_color', 'is_substitute', 'notes', 'rating_raw']].copy()

In [ ]:
# Envoie du dataframe game_player vers la table de la BDD
game_player.to_sql(
    'game_player',
    con=engine,
    if_exists='append',
    index=False)

#### Traitement de la table location

In [ ]:
# Préparation de la table location
location = df_clean[['location']]

In [ ]:
# Envoie du dataframe location vers la table de la BDD
location.to_sql(
    'location',
    con=engine,
    if_exists='append',
    index=False)

#### Traitement de la table game

In [ ]:
# Préparation de la table game
game = df_clean[['game_id', 'game_date', 'ball_type', 'final_score_red', 'final_score_blue', 'table_condition_game', 'referee', 'game_duration', 'winner']]

In [ ]:
# Envoie du dataframe game vers la table de la BDD
game.to_sql(
    'game',
    con=engine,
    if_exists='append',
    index=False)

#### Traitement de la table table_entity

In [ ]:
# Préparation de la table table_entity
table_entity = df_clean['table_id'].drop_duplicates().reset_index(drop=True).to_frame()
table_entity['status'] = None

In [ ]:
# Envoie du dataframe table_entity vers la table de la BDD
table_entity.to_sql(
    'table_entity',
    con=engine,
    if_exists='append',
    index=False)

#### Traitement de la table player

In [ ]:
# Préparation de la table player
player = df_clean[['surname','name','player_age','player_name']]
player['mail'] = None
player['password'] = None
player['role'] = None

In [ ]:
# Envoie du dataframe player vers la table de la BDD
player.to_sql(
    'player',
    con=engine,
    if_exists='append',
    index=False)

## ANALYSE EXPLORATOIRE DES DONNÉS

#### Corrélation entre les buts contre son camp et la défaite

In [179]:
# 1 = défaite pour blue, 0 = pas défaite
df['blue_loss'] = df['winner'].apply(lambda x: 1 if x == 'red' else 0)

In [180]:
# Changement du type en int sur la colonne 'player_own_goals'
df_clean['player_own_goals'] = df_clean['player_own_goals'].astype(int)

In [181]:
# Filtre les lignes avec au moins 1 own goal
own_goal_df = df_clean[df_clean['player_own_goals'] > 0]

# Nombre de défaites quand il y a un own goal
loss_count = (own_goal_df['winner'] == 'blue').sum()

# Nombre total de matchs avec own goal
total_own_goals = len(own_goal_df)

# % de défaites avec own goal
percent_loss = (loss_count / total_own_goals) * 100
print(f"% de défaites avec but contre son camps : {percent_loss:.2f}%")

% de défaites avec but contre son camps : 44.89%


#### Analyse du nombre de win par équipe bleu ou rouge et par table


In [182]:
# Regroupement par table_id et winner
win_counts = df_clean.groupby(['table_id', 'winner']).size().reset_index(name='count')

# Calcule le total de matchs par table
total_matches = win_counts.groupby('table_id')['count'].transform('sum')

# Calcule le pourcentage de victoire
win_counts['win_percent'] = (win_counts['count'] / total_matches) * 100

# On pivote le tableau pour une vue plus lisible
win_percent_table = win_counts.pivot(index='table_id', columns='winner', values='win_percent').fillna(0).reset_index()

win_percent_table


winner,table_id,blue,red,tie
0,T01,44.945189,46.528624,8.526188
1,T02,44.740024,45.828295,9.431681
2,T03,47.950311,40.993789,11.055901
3,T04,44.248826,45.892019,9.859155
4,T05,48.009650,43.305187,8.685163
5,T06,43.178974,47.434293,9.386733
6,T07,47.540984,43.676815,8.782201
7,T08,45.149254,45.895522,8.955224
8,T09,44.815668,47.350230,7.834101
9,T10,44.047619,47.142857,8.809524


#### Analyse du nombre de parties par saison

In [184]:
# Compter le nombre de game_id uniques par saison
games_per_season = df_clean.groupby('season')['game_id'].nunique().reset_index(name='nb_games')
games_per_season

,season,nb_games
0,2023/2024,4971
1,2024/2025,20029
